# OpenStreetMap - get crossing features

In [ ]:
# Standard library and path imports
import set_path

# Third-party library imports
import osmnx as ox
import folium
import geopandas as gpd

# Local or project-specific imports
import plot_utils
import settings as st

if st.my_run == "azure":
    import config_azure as cf
elif st.my_run == "local":
    import config as cf

In [ ]:
if st.my_run == "azure":
    os.system('sudo blobfuse /home/azureuser/cloudfiles/code/blobfuse/sidewalk --tmp-path=/mnt/resource/blobfusetmp --config-file=/home/azureuser/cloudfiles/code/blobfuse/fuse_connection_sidewalk.cfg -o attr_timeout=3600 -o entry_timeout=3600 -o negative_timeout=3600 -o allow_other -o nonempty')

### Get mask of area

In [ ]:
# Import areas
df_areas = gpd.read_file(cf.output_pilot_area)
polygon = df_areas.to_crs(crs=st.CRS_map).unary_union

### Retrieve crossing features

In [ ]:
# Retrieve crossing attributes
tags = {"crossing": True, "footway": 'crossing', "kerb": True, 'railway': ['crossing', 'tram_crossing']}
crossing_features = ox.geometries_from_polygon(polygon, tags)

### Generate map and export crossing features

In [ ]:
# set True for satellite background, False for regular background
satellite = False

# Create Folium map
map = folium.Map(
    location=[52.350547922223434, 4.794019242371844], tiles=plot_utils.generate_map_params(satellite=satellite),
    min_zoom=10, max_zoom=25, zoom_start=17,
    zoom_control=True, control_scale=True, control=False
    )

# Add crossing features
feature_names = crossing_features.columns.tolist()
feature_names.remove('geometry')
tooltip = plot_utils.gen_tooltip(feature_names, feature_names)
geo_j = folium.GeoJson(crossing_features, tooltip=tooltip, style_function=lambda x: {"color": "orange", "weight": 10}).add_to(map)

map

In [ ]:
# Save crossing features as geopackage
crossing_features.to_file(cf.output_osm_crossing_features, driver='GPKG')